In [6]:
from flask import Flask
from flask import request
import json
import re
import string
import pickle
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import tensorflow as tf 
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
app = Flask(__name__)

In [8]:

model = load_model('my_model.h5')
ps = PorterStemmer()

file = open('processed_list_of_tweets.pkl', 'rb')
text_data = pickle.load(file)
file.close()

x = (int)(len(text_data) * 0.8)
training_text = text_data[0:x]

tokenizer = Tokenizer(num_words = 5000, oov_token= '<<none>>')
tokenizer.fit_on_texts(training_text)
word_index = tokenizer.word_index

def words_only(text):
    '''Remove puctuations, underscores, numbers, words containing numbers and @_mentions.'''
    text = text.lower()
    text = re.sub(r'\w*\#angry\w*', '', text)
    text = re.sub(r'\w*\#hate\w*', '', text)
    text = re.sub(r'\w*\@\w*', '', text)
    text = re.sub(r'[^\w\s]','', text)
    text = re.sub(r'\_','',text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = re.sub(' +', ' ',text)
    return remove_stop_words(text.strip())

#nltk.download('punkt')

def remove_stop_words(text):
    '''Remove stop words and stem the words'''
    text_filtered = ""
    text = word_tokenize(text)
    for word in text:
        if word not in stopwords.words("english"):
            word = ps.stem(word)
            text_filtered = text_filtered+" "+word
    return text_filtered.strip()

def predict_the_sentiment(text):
    txt2 = words_only(text)
    txt = []
    txt.append(txt2)
    seq = tokenizer.texts_to_sequences(txt)
    padded = pad_sequences(seq, maxlen=50, padding='post', truncating='post')

    pred = model.predict(padded)
    labels = ['happy', 'sad', 'angry', 'hate', 'neutral'] 
    #print(pred)
    #print(np.argmax(pred))
    return(labels[np.argmax(pred)-1])

In [9]:
@app.route('/analysis', methods=['POST'])
def analysis_data():
    if (request.method == 'POST'):
        data = json.loads(request.data)
        my_data = str(data['data'])
        sentiment = predict_the_sentiment(my_data)
#         print(str(sentiment))
        return my_data
    else:
        return 'error'
if __name__ == '__main__':
    app.run('localhost','7001')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:7001/ (Press CTRL+C to quit)
[2020-07-12 23:57:20,270] ERROR in app: Exception on /analysis [POST]
Traceback (most recent call last):
  File "C:\Users\Hassan Sheikh\.conda\envs\fyp\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Hassan Sheikh\.conda\envs\fyp\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Hassan Sheikh\.conda\envs\fyp\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Hassan Sheikh\.conda\envs\fyp\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Hassan Sheikh\.conda\envs\fyp\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Hassan Sheikh\.conda\envs\fyp\lib\site-packages\flask\app.py", line 1936, in dispatch_request
